In [ ]:
# %% [markdown]
# # Animated Visualization of the Synaptic Quantum Nanoreactor
# 
# This notebook creates an animation showing how Posner molecules form in the synapse

# %%
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle, Rectangle, FancyBboxPatch, Wedge
from matplotlib.collections import PatchCollection
import matplotlib.patheffects as path_effects
from IPython.display import HTML
from matplotlib import colors

# Set up nice plotting
plt.style.use('dark_background')
%matplotlib inline

# %% [markdown]
# ## Create the Animation

# %%
class SynapticNanoreactorAnimation:
    def __init__(self):
        # Set up the figure
        self.fig = plt.figure(figsize=(16, 10))
        
        # Create subplots
        gs = self.fig.add_gridspec(2, 3, width_ratios=[2, 1, 1], height_ratios=[1, 1])
        self.ax_main = self.fig.add_subplot(gs[:, 0])  # Main animation
        self.ax_ca = self.fig.add_subplot(gs[0, 1])    # Calcium dynamics
        self.ax_posner = self.fig.add_subplot(gs[1, 1]) # Posner accumulation
        self.ax_quantum = self.fig.add_subplot(gs[0, 2]) # Quantum coherence
        self.ax_enhance = self.fig.add_subplot(gs[1, 2]) # Enhancement factors
        
        # Animation parameters
        self.n_frames = 200
        self.time = np.linspace(0, 2, self.n_frames)  # 2 seconds
        
        # Synaptic parameters (in nm)
        self.cleft_width = 20
        self.width = 400
        self.channel_positions = [(100, 5), (200, 5), (300, 5)]
        self.template_positions = [(150, 10), (250, 10)]
        
        # Initialize elements
        self.setup_main_view()
        self.setup_plots()
        
        # Data storage
        self.ca_history = []
        self.posner_history = []
        self.coherence_history = []
        
    def setup_main_view(self):
        """Set up the main synaptic view"""
        self.ax_main.set_xlim(0, self.width)
        self.ax_main.set_ylim(-5, 30)
        self.ax_main.set_aspect('equal')
        self.ax_main.set_title('Synaptic Quantum Nanoreactor', fontsize=20, pad=20)
        self.ax_main.set_xlabel('Distance (nm)', fontsize=14)
        
        # Hide y-axis
        self.ax_main.set_yticks([])
        
        # Add labels on the side
        self.ax_main.text(-20, 0, 'Presynaptic', rotation=90, va='center', fontsize=12)
        self.ax_main.text(-20, 20, 'Cleft', rotation=90, va='center', fontsize=12)
        self.ax_main.text(-20, 25, 'Postsynaptic', rotation=90, va='center', fontsize=12)
        
        # Draw membranes
        pre_membrane = Rectangle((0, -5), self.width, 5, facecolor='#2E86AB', 
                                edgecolor='white', linewidth=2)
        post_membrane = Rectangle((0, 25), self.width, 5, facecolor='#A23B72', 
                                 edgecolor='white', linewidth=2)
        self.ax_main.add_patch(pre_membrane)
        self.ax_main.add_patch(post_membrane)
        
        # Draw cleft
        cleft = Rectangle((0, 0), self.width, 20, facecolor='#0A0A0A', 
                         alpha=0.5, edgecolor='none')
        self.ax_main.add_patch(cleft)
        
        # Add channels (will be animated)
        self.channels = []
        self.ca_streams = []
        for x, y in self.channel_positions:
            # Channel
            channel = FancyBboxPatch((x-10, -5), 20, 10, 
                                    boxstyle="round,pad=2",
                                    facecolor='#F18F01', 
                                    edgecolor='white',
                                    linewidth=2)
            self.ax_main.add_patch(channel)
            self.channels.append(channel)
            
            # Calcium stream (initially invisible)
            stream = Wedge((x, 5), 30, 45, 135, 
                          facecolor='cyan', alpha=0, 
                          edgecolor='none')
            self.ax_main.add_patch(stream)
            self.ca_streams.append(stream)
        
        # Add templates
        self.templates = []
        for x, y in self.template_positions:
            template = Circle((x, y), 8, facecolor='#C73E1D', 
                            edgecolor='white', linewidth=2)
            self.ax_main.add_patch(template)
            self.templates.append(template)
            
            # Add label
            text = self.ax_main.text(x, y, 'T', ha='center', va='center', 
                                   fontsize=10, fontweight='bold', color='white')
            text.set_path_effects([path_effects.withStroke(linewidth=2, foreground='black')])
        
        # Posner molecules (will be animated)
        self.posner_molecules = []
        
        # Add legend elements
        legend_elements = [
            plt.Line2D([0], [0], marker='s', color='w', markerfacecolor='#F18F01', 
                      markersize=10, label='Ca²⁺ Channel'),
            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#C73E1D', 
                      markersize=10, label='Template Site'),
            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', 
                      markersize=8, label='Posner Molecule'),
            plt.Line2D([0], [0], color='cyan', linewidth=3, alpha=0.7, 
                      label='Ca²⁺ Microdomain')
        ]
        self.ax_main.legend(handles=legend_elements, loc='upper right', 
                           framealpha=0.8, fontsize=10)
        
    def setup_plots(self):
        """Set up the data plots"""
        # Calcium dynamics plot
        self.ax_ca.set_title('Calcium Dynamics', fontsize=14)
        self.ax_ca.set_xlabel('Time (s)')
        self.ax_ca.set_ylabel('[Ca²⁺] (μM)')
        self.ax_ca.set_xlim(0, 2)
        self.ax_ca.set_ylim(0, 50)
        self.ca_line, = self.ax_ca.plot([], [], 'c-', linewidth=2)
        self.ax_ca.axhline(0.1, color='gray', linestyle='--', alpha=0.5, 
                          label='Baseline')
        
        # Posner accumulation plot
        self.ax_posner.set_title('Posner Formation', fontsize=14)
        self.ax_posner.set_xlabel('Time (s)')
        self.ax_posner.set_ylabel('[Posner] (nM)')
        self.ax_posner.set_xlim(0, 2)
        self.ax_posner.set_ylim(0, 100)
        self.posner_line, = self.ax_posner.plot([], [], 'y-', linewidth=2)
        
        # Quantum coherence plot
        self.ax_quantum.set_title('Quantum Coherence', fontsize=14)
        self.ax_quantum.set_xlabel('Time (s)')
        self.ax_quantum.set_ylabel('Coherence Time (ms)')
        self.ax_quantum.set_xlim(0, 2)
        self.ax_quantum.set_ylim(0, 1000)
        self.coherence_line, = self.ax_quantum.plot([], [], 'g-', linewidth=2)
        self.ax_quantum.axhline(20, color='red', linestyle='--', alpha=0.5,
                               label='STDP window')
        
        # Enhancement factor bar chart
        self.ax_enhance.set_title('Enhancement Factors', fontsize=14)
        factors = ['Base', 'Micro-\ndomain', 'Template', 'Electro-\nstatic', '2D\nConfine']
        self.enhance_values = [1, 1, 1, 1, 1]
        self.enhance_bars = self.ax_enhance.bar(factors, self.enhance_values, 
                                               color=['gray', 'cyan', 'red', 'orange', 'green'])
        self.ax_enhance.set_ylabel('Enhancement (×)')
        self.ax_enhance.set_yscale('log')
        self.ax_enhance.set_ylim(0.1, 1e6)
        
        # Style all plots
        for ax in [self.ax_ca, self.ax_posner, self.ax_quantum, self.ax_enhance]:
            ax.grid(True, alpha=0.2)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
    
    def init_animation(self):
        """Initialize animation"""
        return (self.ca_line, self.posner_line, self.coherence_line, 
                *self.ca_streams, *self.enhance_bars)
    
    def animate(self, frame):
        """Animation function"""
        t = self.time[frame]
        
        # Spike events (every 0.2s)
        spike_times = np.arange(0.1, 2, 0.2)
        is_spiking = any(abs(t - st) < 0.05 for st in spike_times)
        
        # 1. Animate calcium channels and microdomains
        for i, (channel, stream) in enumerate(zip(self.channels, self.ca_streams)):
            if is_spiking:
                # Open channel - change color
                channel.set_facecolor('#FFD23F')
                # Show calcium stream
                stream.set_alpha(0.7 - 0.5 * (t % 0.2) / 0.2)  # Fade effect
                stream.set_radius(30 + 20 * (t % 0.2) / 0.2)  # Expanding
            else:
                # Closed channel
                channel.set_facecolor('#F18F01')
                stream.set_alpha(0)
        
        # 2. Update calcium concentration
        ca_local = 0.1  # baseline
        if is_spiking:
            ca_local = 20 + 20 * np.exp(-(t % 0.2) / 0.05)  # Spike decay
        self.ca_history.append(ca_local)
        if len(self.ca_history) > frame + 1:
            self.ca_history = self.ca_history[:frame + 1]
        
        times = self.time[:len(self.ca_history)]
        self.ca_line.set_data(times, self.ca_history)
        
        # 3. Posner formation (accumulates with spikes)
        n_spikes = sum(t > st for st in spike_times)
        posner_conc = min(100, n_spikes * 10 * (1 - np.exp(-t)))  # Saturating growth
        self.posner_history.append(posner_conc)
        if len(self.posner_history) > frame + 1:
            self.posner_history = self.posner_history[:frame + 1]
        
        self.posner_line.set_data(times, self.posner_history)
        
        # 4. Add/update Posner molecules in main view
        n_posner_visible = int(posner_conc / 10)  # 1 molecule per 10 nM
        
        # Remove extra molecules
        while len(self.posner_molecules) > n_posner_visible:
            mol = self.posner_molecules.pop()
            mol.remove()
        
        # Add new molecules near templates
        while len(self.posner_molecules) < n_posner_visible:
            # Choose position near a template
            template_idx = len(self.posner_molecules) % len(self.template_positions)
            tx, ty = self.template_positions[template_idx]
            
            # Add some randomness
            x = tx + np.random.normal(0, 10)
            y = ty + np.random.normal(0, 3)
            
            mol = Circle((x, y), 3, facecolor='yellow', 
                        edgecolor='orange', linewidth=1.5)
            self.ax_main.add_patch(mol)
            self.posner_molecules.append(mol)
        
        # 5. Quantum coherence (depends on Posner concentration)
        if posner_conc > 0:
            # P31: 1000 ms at low conc, decreasing with concentration
            coherence = 1000 / (1 + posner_conc / 100)
        else:
            coherence = 0
        self.coherence_history.append(coherence)
        if len(self.coherence_history) > frame + 1:
            self.coherence_history = self.coherence_history[:frame + 1]
        
        self.coherence_line.set_data(times, self.coherence_history)
        
        # 6. Update enhancement factors
        if is_spiking:
            self.enhance_values = [1, 10000, 10, 3, 5]  # Full enhancement
        else:
            self.enhance_values = [1, 10, 1, 1, 1]  # Partial enhancement
        
        cumulative = np.cumprod(self.enhance_values)
        for bar, val in zip(self.enhance_bars, cumulative):
            bar.set_height(val)
        
        # Add cumulative value on top bar
        self.ax_enhance.texts.clear()
        for i, (bar, val) in enumerate(zip(self.enhance_bars, cumulative)):
            if i == len(self.enhance_bars) - 1:  # Last bar
                self.ax_enhance.text(bar.get_x() + bar.get_width()/2, 
                                   val * 1.5, f'{val:,.0f}×', 
                                   ha='center', fontweight='bold', fontsize=12)
        
        # 7. Add time indicator
        self.fig.suptitle(f'Synaptic Quantum Nanoreactor - Time: {t:.2f}s', 
                         fontsize=20, y=0.98)
        
        return (self.ca_line, self.posner_line, self.coherence_line, 
                *self.ca_streams, *self.enhance_bars, *self.posner_molecules)

# %% [markdown]
# ## Create and Display the Animation

# %%
# Create the animation
nanoreactor = SynapticNanoreactorAnimation()

# Create animation object
anim = animation.FuncAnimation(nanoreactor.fig, nanoreactor.animate, 
                             init_func=nanoreactor.init_animation,
                             frames=nanoreactor.n_frames, 
                             interval=50,  # 50ms per frame = 20 fps
                             blit=False,
                             repeat=True)

# Display in notebook
plt.close()  # Close the static figure
HTML(anim.to_jshtml())

# %% [markdown]
# ## Save Animation Options

# %%
# To save as video file (requires ffmpeg):
# anim.save('synaptic_nanoreactor.mp4', writer='ffmpeg', fps=20, dpi=150)

# To save as GIF (requires pillow or imagemagick):
# anim.save('synaptic_nanoreactor.gif', writer='pillow', fps=20, dpi=100)

print("Animation created! Use the controls to play/pause/scrub through the animation.")

# %% [markdown]
# ## Animation Explanation
# 
# ### What the Animation Shows:
# 
# 1. **Main Panel (Left)**: 
#    - Blue = presynaptic membrane with Ca²⁺ channels
#    - Pink = postsynaptic membrane  
#    - Channels turn yellow when opening
#    - Cyan wedges = calcium microdomains
#    - Red circles = template nucleation sites
#    - Yellow dots = Posner molecules forming
# 
# 2. **Calcium Dynamics (Top Right)**:
#    - Shows local [Ca²⁺] spiking to 40 μM during channel opening
#    - Baseline is 0.1 μM
# 
# 3. **Posner Formation (Middle Right)**:
#    - Accumulates with each spike
#    - Reaches ~100 nM after multiple spikes
# 
# 4. **Quantum Coherence (Top Far Right)**:
#    - Shows T₂ coherence time 
#    - Starts high, decreases as Posner accumulates
#    - Red line = 20 ms STDP window
# 
# 5. **Enhancement Factors (Bottom Right)**:
#    - Shows cumulative enhancement during activity
#    - Reaches 1.5 million × during spikes!
# 
# ### Key Insights Visualized:
# 
# - **Spatial organization**: Posner forms near template sites
# - **Temporal dynamics**: Accumulation over multiple spikes  
# - **Quantum window**: High coherence when Posner first forms
# - **Enhancement cascade**: Multiple factors multiply together

# %%
# Create a static summary figure showing key frames
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Key Stages of Posner Formation in Synaptic Nanoreactor', fontsize=16)

# Define key timepoints
timepoints = [
    (0, "Resting State"),
    (0.1, "First Spike - Channels Open"),
    (0.5, "Multiple Spikes - Posner Accumulating"),
    (1.0, "Sustained Activity - High Posner"),
    (1.5, "Quantum Enhancement Window"),
    (2.0, "Final State")
]

# For each timepoint, show a snapshot
for idx, (t, title) in enumerate(timepoints):
    ax = axes[idx // 3, idx % 3]
    ax.set_title(title, fontsize=12)
    ax.set_xlim(0, 400)
    ax.set_ylim(-5, 30)
    
    # Draw basic structure
    ax.add_patch(Rectangle((0, -5), 400, 5, facecolor='#2E86AB'))
    ax.add_patch(Rectangle((0, 25), 400, 5, facecolor='#A23B72'))
    
    # Show state at this timepoint
    is_spiking = any(abs(t - st) < 0.05 for st in np.arange(0.1, 2, 0.2))
    n_spikes = sum(t > st for st in np.arange(0.1, 2, 0.2))
    
    # Channels
    for x, y in [(100, 5), (200, 5), (300, 5)]:
        color = '#FFD23F' if is_spiking else '#F18F01'
        ax.add_patch(Rectangle((x-10, -5), 20, 10, facecolor=color))
        
        if is_spiking:
            # Show microdomain
            circle = Circle((x, 5), 30, facecolor='cyan', alpha=0.3)
            ax.add_patch(circle)
    
    # Templates
    for x, y in [(150, 10), (250, 10)]:
        ax.add_patch(Circle((x, y), 8, facecolor='#C73E1D'))
    
    # Posner molecules
    n_posner = min(10, n_spikes * 2)
    for i in range(n_posner):
        x = 150 + (i % 2) * 100 + np.random.uniform(-20, 20)
        y = 10 + np.random.uniform(-5, 5)
        ax.add_patch(Circle((x, y), 3, facecolor='yellow'))
    
    # Add annotations
    ax.text(200, -10, f"t = {t:.1f}s", ha='center', fontsize=10)
    ax.text(200, 32, f"{n_spikes} spikes", ha='center', fontsize=10)
    
    ax.set_aspect('equal')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\nAnimation complete! This shows how the synaptic nanoreactor transforms")
print("neural activity into quantum resources for enhanced computation.")